# 2018 청룡영화제 남녀주연상 수상자 예측 
2013-2017 5년간의 청룡영화제 남녀주연상 수상작 댓글 분석을 통해 2018 수상자 예측하기 

> -가설1) 이전 수상작 댓글에서 연기력,배우와 관련된 단어들이 많이 언급될 것이다.
>
> -가설2) 가설1에서 도출한 단어들이 많이 언급된 영화의 배우가 2018 청룡영화제 남녀주연상을 수상할것이다.

## 1.데이터 수집

- 2018 영화흥행순 70개 영화 api 다운  (영화진흥위원회)
- 2013-2017 수상작 데이터 (csv로) 
- 리뷰 크롤링 

In [3]:
#리뷰 크롤링을 위한 모듈 임포트 
from bs4 import BeautifulSoup
from selenium import webdriver
from IPython.display import Image
from urllib.request import urlopen
import pandas as pd
import re
import time

### 리뷰크롤링모듈 만들기 

In [4]:

# 웹드라이브로 크롬브라우즈 띄운다.
class OpenDriverMovieSite():
    driver_path = "../driver/chromedriver.exe"
    driver = webdriver.Chrome(executable_path=driver_path)
    url_page = 'https://movie.naver.com/movie/point/af/list.nhn'
    movieName = ''
    allReviews = []
    allReviewsIndex = []
    a = 1

    def __init__(self):
        # 클래스를 실행하면 홈페이지로 이동한다.
        self.driver.get(self.url_page)
        time.sleep(1)

    def ReadCSV(self, movie2018):
        movie_name = movie2018['영화명']
        movie_name[1]
        for i in range(0, len(movie2018)):  # CSV 파일의 Rows 갯수를 뒤에 넣어주세여 (1, Rows갯수)
            #             print(self.movieName)
            print(movie_name[i])
            self.movieName = movie_name[i]
            self.BasicOpen()
            self.InputMovieTitle(movie_name[i])

    def BasicOpen(self):
        # 켜진 홈페이지에서 관련영화를 클릭한 후 새로운 창으로 전환하는 것까지
        # 영화명이 지속적으로 바뀔 것이므로 BasicOpen 함수를 활용
        self.driver.find_element_by_xpath('//*[@id="old_content"]/fieldset/form/select').click()
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="old_content"]/fieldset/form/select/option[2]').click()
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="old_content"]/fieldset/form/input[3]').click()
        time.sleep(0.5)
        self.driver.switch_to.window(self.driver.window_handles[-1])
        self.driver.find_element_by_xpath(
            '//*[@id="page_content"]/div/div/div/div/form/table/tbody/tr/td/input[1]').click()
        time.sleep(0.5)

    def InputMovieTitle(self, movie_title):
        self.driver.find_element_by_css_selector(
            '#page_content > div > div > div > div > form > table > tbody > tr > td > input.input_type_text_1').send_keys(
            movie_title)
        time.sleep(0.5)
        self.driver.find_element_by_xpath(
            '//*[@id="page_content"]/div/div/div/div/form/table/tbody/tr/td/input[2]').click()
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="page_content"]/div/div/div[2]/table/tbody/tr[2]/td/a/img').click()
        time.sleep(0.5)
        self.driver.switch_to.window(self.driver.window_handles[0])
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="old_content"]/fieldset/form/input[4]').click()

        html = urlopen(self.driver.current_url)
        soup = BeautifulSoup(html, "lxml")
        # 코드값 찾아주기
        pattern = re.compile("\d+")
        result = pattern.findall(self.driver.current_url)
        print(result)
        code = result[0] if len(result) > 0 else ''

        # 리뷰의 총개수
        num = soup.find("strong", "c_88 fs_11").get_text()
        num_total = int(num.replace(',', ''))
        last_page = num_total // 10 + 1
        temp = "번호 : {} // 영화명 : {} // code : {} // num_total : {} // last_page : {}".format(self.a, self.movieName,
                                                                                             code, num_total, last_page)
        self.allReviewsIndex.append(temp)
        #         print("번호 : {} // 영화명 : {} // code : {} // num_total : {} // last_page : {}".format(self.a, self.movieName, code, num_total, last_page))
        self.a = self.a + 1

        #         last_page = 1000 if last_page > 1000 else last_page
        last_page = 1000 if last_page > 1000 else last_page

        self.GetReviews(code, num_total, last_page)

    def GetReviews(self, code, num_total, last_page):
        tmp = []
        for i in range(last_page):

            # 각 리뷰 페이지로 이동해서 그 url로 드라이버 창 넘어가고-> 수프화
            url_page = 'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword={}&target=&page={}'.format(code,
                                                                                                                  i + 1)
            print("\"{}\"의 {} 페이지를 수집하고 있습니다..".format(self.movieName, url_page))
            html = urlopen(url_page)
            soup = BeautifulSoup(html, "lxml")
            # 리뷰들 뽑아줌
            review = soup.find_all("td", "title")
            for i in range(len(review)):
                review[i] = review[i].get_text()
                review[i] = review[i].replace('\n', '')
                review[i] = review[i].replace(self.movieName, '')  # 왜 영화명을 '' 처리하는지 잘 모르겠지만 우선 둠
                review[i] = review[i].replace('\r', '')
                review[i] = review[i].replace('\t', '')
                review[i] = review[i].replace('신고', '')
                tmp.append(review[i])
        self.allReviews.append(tmp)


if __name__ == "__main__":
    opendrivermoviesite = OpenDriverMovieSite()
    movie2018 = pd.read_csv('data/awarded.csv', sep=',', encoding='euc-kr')
    opendrivermoviesite.ReadCSV(movie2018)

택시운전사
['146469']
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=1 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=2 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=3 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=4 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=5 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=6 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=7 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=8 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=9 페

"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=75 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=76 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=77 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=78 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=79 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=80 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=81 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=82 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=83 페이지를 수집하고

"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=148 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=149 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=150 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=151 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=152 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=153 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=154 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=155 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=156 

"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=221 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=222 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=223 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=224 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=225 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=226 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=227 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=228 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=229 

"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=294 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=295 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=296 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=297 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=298 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=299 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=300 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=301 페이지를 수집하고 있습니다..
"택시운전사"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=146469&target=&page=302 

...

"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=764 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=765 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=766 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=767 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=768 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=769 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=770 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=771 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=772 페이지를 수집하고

"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=838 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=839 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=840 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=841 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=842 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=843 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=844 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=845 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=846 페이지를 수집하고

"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=912 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=913 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=914 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=915 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=916 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=917 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=918 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=919 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=920 페이지를 수집하고

"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=986 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=987 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=988 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=989 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=990 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=991 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=992 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=993 페이지를 수집하고 있습니다..
"내부자들"의 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=121788&target=&page=994 페이지를 수집하고

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=62123): Max retries exceeded with url: /session/dc3400a108bbd01105dc92883ec8bd4e/element (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000000009261C18>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다',))

In [ ]:
#데이터 프레임으로 변환해서  csv 파일로 저장
df_all = pd.DataFrame(opendrivermoviesite.allReviews)
df_all.to_csv('data/allreviews.csv', sep=',', encoding='euc-kr')
allreview_index.to_csv('data/allreview_index_df.csv', sep=',', encoding='euc-kr')

In [ ]:
#타이틀, 코드, 총 리뷰수, 마지막 페이지 정보 분리
title = []
code = []
num_total= []
last_page = []
for i in range(len(opendrivermoviesite.allReviewsIndex)):
    y = opendrivermoviesite.allReviewsIndex[i].split("//")
    t = y[1].split(":")
    c = y[2].split(":")
    n = y[3].split(":")
    l = y[4].split(":")
    title.append(t[1])
    code.append(c[1])
    num_total.append(n[1])
    last_page.append(l[1])
title[:3] , code[:3],  num_total[:3], last_page[:3]

In [ ]:
#데이터 프레임으로 변환
allreview_index = pd.DataFrame(({'제목':title, 
                          '코드':code, 
                          '총리뷰수':num_total, 
                          '마지막페이지':last_page}))

In [ ]:
allreview_index

In [ ]:
#csv 파일로 저장
allreview_index.to_csv('data/allreview_index_df.csv', sep=',', encoding='euc-kr')

## 2.데이터전처리 
-명사추출모듈

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud

from konlpy.corpus import kolaw
from konlpy.tag import Twitter
twitter = Twitter()

In [ ]:
# 모든 댓글 합쳐진문장가지고옴 - > 리스트로 영화별로 파싱 ->  각 영화별에 담긴 댓글 단어 탑 100으로 
def df_review(filepath):
    #파일다듬기 
    f = open(filepath, 'r')
    review_raw = f.read()
    f.close()
    review_raw_split=review_raw.split("\n")
    
    # 명사 top 100추출 후 result 에 저장 
    result =[]
    for i in review_raw_split:
        nouns = twitter.nouns(i)
        nouns_nltk = nltk.Text(nouns, name ='리뷰')
        result.append(nouns_nltk.vocab().most_common(100))
        j+=1   
#     #데이터프레임화해서 저장
#     df = pd.DataFrame({"명사_숫자":result})
#     df.to_csv("df_review.csv", sep=",", encoding="utf-8")

In [ ]:
filepath = 'data/_2018_awarded_reviews.csv'
f = open(filepath, 'r', encoding = "UTF-8")
review_raw = f.read()
f.close()
review_raw

In [ ]:
len(review_raw.split("\n")[1].split(","))

In [ ]:
# filepath = 'data/_2018_awarded_reviews.csv'
# f = open(filepath, 'r', encoding = "UTF-8")
# review_raw = f.read()
# f.close()
# review_raw_split=review_raw.split("\n")
                                               
    
#     # 명사 top 100추출
# result =[]
# for i in review_raw_split:
#     nouns = twitter.nouns(i)
#     nouns_nltk = nltk.Text(nouns, name ='리뷰')
#     result.append(nouns_nltk.vocab().most_common(100))
    
#     #n개 영화 noun_count 뽑아서 저장
# review_nouns_count= []
# j = 0
# for i in result: 
#     review_nouns_count.append(i)
#     print("{}번째 영화의 형태수 빈도수를 추출중입니다".format(j))
#     j+=1
    
#     #데이터프레임화해서 저장
# df = pd.DataFrame({"명사_숫자":review_nouns_count})
# df

In [ ]:
#해당 키워드가 있는지 확인 
templist = []
def tokenfinder(arg1, arg2):
    for i in range(len(df['명사_숫자'])):
         for j in range(len(arg1.iloc[i][0])):
             if arg1.iloc[i]['명사_숫자'][j][0] in arg2:
                 templist.append(arg1.iloc[i]['명사_숫자'][j])
tokenfinder(df, '역사')
templist


In [ ]:
#인덱스 바꿔주기
df_index = pd.read_csv("data/_2018_awarded_movies.csv")
df_index2 =df_index["영화명"]

In [ ]:
# actors = ['한지민', '이태리', '김민희', '나문희', '박보영', '하정우', '주지훈']

In [ ]:
filepath = 'allreviews_plusname_test.csv'
df_review(filepath)